In [1]:
import pandas as pd
from collections import defaultdict
import math
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
import numpy as np
from str_helper import *
import pickle 

# Populate definitions dict

apa = pd.read_csv('ontologies\APAONTO.csv').fillna('') #Preferred Label, Definitions, Alt name
mf = pd.read_csv('ontologies\MF.csv').fillna('')
mfo = pd.read_csv('ontologies\MFOMD.csv').fillna('')
sui = pd.read_csv('ontologies\suicideo.csv').fillna('')


thes=defaultdict(dict)

def add_defn(w, dfn, onto):
    if w=='':
        pass
    d=thes[w]
    d[onto]=re.sub(r'\[wikipedia: http\S+\]', '', dfn.replace('\n', ' '), flags=re.IGNORECASE)
    thes[w] = d


for row in apa.iterrows():
    word = row[1]['Preferred Label']
    defn = row[1]['Definitions']
    if defn=='':
        continue
    syn = [word]+row[1]['Alt name'].split('|')
    for w in syn:
        add_defn(w, defn, 'apa')    
        
for row in mf.iterrows():
    word = row[1]['Preferred Label']
    defn = row[1]['Definitions']
    if defn=='':
        continue
    add_defn(word, defn, 'mf')
    
for row in mfo.iterrows():
    word = row[1]['Preferred Label']
    defn = row[1]['Definitions']
    if defn=='':
        continue
    syn = [word]+row[1]['Synonyms'].split('|')
    for w in syn:
        add_defn(w, defn, 'mfo')

for row in sui.iterrows():
    word = row[1]['Preferred Label']
    defn = row[1]['Definitions']
    if defn=='':
        continue
    syn = [word]+row[1]['alternative term'].split('\n')
    for w in syn:
        add_defn(w, defn, 'sui')
        

with open('psychThesaurusDict.pkl','wb') as f:
        pickle.dump(thes, f)      

## Vocab for hyperspace training - psychlopedia

In [2]:
window_size = 10


with open('psychThesaurusDict.pkl','rb') as f:
        thes = pickle.load(f)      

def get_studded_defn(w, dfn):
    
    w=clean_sent(w.lower())
    stud=[w]
    dfn=clean_sent(dfn)
    tok = word_tokenize(dfn)
    for i in range(0, len(tok), window_size):
        stud += tok[i:i+window_size]+[w]
    return ' '.join(stud)

def dict_iter():
    for w,d in thes.items():
        dfns = d.values()
        for dd in dfns:
            if len(w)<2:
                continue
            yield get_studded_defn(w,dd)

gen=dict_iter()
with open('psychlopedia_train_verbose.txt','w') as f:
    for l in gen:
        f.write(l+'\n')
        


ADHD ['Attention', 'deficit', 'hyperactivity', 'disorder']
(ADHD) Attention deficit hyperactivity disorder
ADD ['Attention', 'deficit', 'disorder"']
(ADD) Attention deficit disorder"
NHS ['National', 'Health', 'Service']
(NHS) National Health Service
PER ['so', 'named', 'Period']
TIM ['Period', 'PER', 'Timeless']
CRY ['Timeless', 'TIM', 'Cryptochrome']
PER ['so', 'named', 'Period']
TIM ['Period', 'PER', 'Timeless']
CRY ['Timeless', 'TIM', 'Cryptochrome']
Pavlovian ['classical']
LEARNING ['the', 'aid', 'of', 'external', 'cues', 'Compare', 'SERIAL', 'ANTICIPATION']
LEARNING ['the', 'aid', 'of', 'external', 'cues', 'Compare', 'SERIAL', 'ANTICIPATION']
LEARNING ['to', 'a', 'new', 'context', 'or', 'setting', 'Compare', 'TRANSFER']
TESTING ['established', 'scales', 'or', 'standards', 'Consider', 'also', 'SCORING']
ATP ['to', 'form', 'triphosphates']
ATP ['to', 'form', 'triphosphates']
EVENTS ['change', 'due', 'to', 'illness', 'Compare', 'EXPERIENCES']
EVENTS ['change', 'due', 'to', 'illness'

In [3]:
def split_words(abst):
    return re.sub(r'([a-zA-Z])[^a-zA-Z]+([a-zA-Z])', r'\1 \2', abst)

split_words('MGI.|Attention')

'MGI Attention'